In [ ]:
import pandas as pd
# Importar biblioteca biqquery
from google.cloud import bigquery

# Bibliotecas Google Sheet
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)
from gspread_dataframe import set_with_dataframe

# Montar Drive
from google.colab import drive
drive.mount('/content/drive')

# Classe dados 
!cp /content/drive/MyDrive/Colab_Notebooks/pacotes_modulos/modulos_colab.py /content
from modulos_colab import dados
# Chaves e tokens
!cp /content/drive/MyDrive/Colab_Notebooks/Autenticacoes/chaves_tokens.py /content
%run chaves_tokens.py


Mounted at /content/drive


# Relatório Repasse

In [ ]:
rel_repasse_1 = pd.read_excel('/content/repasse-adidas-2023-05-20-2023-05-26.xlsx')
#rel_repasse_2 = pd.read_excel('/content/repasse-adidas-14032023-2.xlsx')
rel_repasse_total = pd.concat([rel_repasse_1])
rel_repasse_total['% Serviços de Marketplace'] = round((rel_repasse_total['% Serviços de Marketplace']/100),3)


rel_repasse_estorno = rel_repasse_total.query("`Método de pagamento`=='Estorno'")



din_rel_repasse_estorno = pd.DataFrame(pd.pivot_table(rel_repasse_estorno, index='ID do pedido Seller', values='Valor líquido da parcela', aggfunc='sum'))
din_rel_repasse_estorno.reset_index(inplace=True)
din_rel_repasse_estorno.rename(columns={'Valor líquido da parcela':'Estorno todas parcelas'}, inplace=True)


repasse_final = pd.merge(rel_repasse_total, din_rel_repasse_estorno, how='left', on='ID do pedido Seller')


def calculo_ajuste_todas_parcelas(repasse_final):
    if repasse_final['Método de pagamento'] == 'Estorno':
        return repasse_final['Estorno todas parcelas']
    else:
        return 0.00
repasse_final['Estorno todas parcelas'] = repasse_final.apply(calculo_ajuste_todas_parcelas, axis=1)



def calculo_produto(repasse_final):
    if repasse_final['Método de pagamento'] == 'Estorno':
        return round((repasse_final['Estorno todas parcelas']*(repasse_final['% Serviços de Marketplace']-0.065))+(repasse_final['Estorno todas parcelas']),2)
    else:
        return 0.00
repasse_final['Custo do produto'] = repasse_final.apply(calculo_produto, axis=1)




def calculo_mdr(repasse_final):
    if repasse_final['Método de pagamento'] == 'Estorno':
        return round((repasse_final['Custo do produto']*0.065),2)
    else:
        return 0.00
repasse_final['MDR do produto'] = repasse_final.apply(calculo_mdr, axis=1)




def calculo_comissao(repasse_final):
    if repasse_final['Método de pagamento'] == 'Estorno':
        return round((repasse_final['Custo do produto']*repasse_final['% Serviços de Marketplace']),2)
    else:
        return 0.00
repasse_final['Custo de comissão (produto)'] = repasse_final.apply(calculo_comissao, axis=1)



def calculo_liq_produto(repasse_final):
    if repasse_final['Método de pagamento'] == 'Estorno':
        return round((repasse_final['Custo do produto']-repasse_final['Custo de comissão (produto)']-repasse_final['Valor fixo por pedido']),2)
    else:
        return 0.00
repasse_final['Valor líquido do produto'] = repasse_final.apply(calculo_liq_produto, axis=1)



def calculo_liq_produto_mdr(repasse_final):
    if repasse_final['Método de pagamento'] == 'Estorno':
        return round((repasse_final['Valor líquido do produto']+repasse_final['MDR do produto']),2)
    else:
        return 0.00
repasse_final['Valor líquido do produto + MDR'] = repasse_final.apply(calculo_liq_produto_mdr, axis=1)



def calculo_parcela(repasse_final):
    if repasse_final['Método de pagamento'] == 'Estorno' and ((repasse_final['Valor líquido da parcela']) > (repasse_final['Valor líquido do produto + MDR']*0.9)):
        return round((repasse_final['Valor líquido do produto + MDR']),2)
    elif repasse_final['Método de pagamento'] == 'Estorno' and ((repasse_final['Valor líquido da parcela']) < (repasse_final['Valor líquido do produto + MDR']*0.9)):
        return round((repasse_final['Valor líquido do produto + MDR']/repasse_final['Total de parcelas']),2)
    else:
        return 0.00
repasse_final['Valor estornado por parcela'] = repasse_final.apply(calculo_parcela, axis=1)

repasse_final.reset_index(inplace=True)

repasse_final.to_csv("/content/drive/MyDrive/dados_repasse/relatorio_repasse_26052023.csv", encoding='cp1252')

#dados.grava_tabela_drive('conferencia_repasse', 'Página1', repasse_final)

print("Tabela Atualizada!")




Tabela Atualizada!
